In [1]:
import numpy as np
import faiss
import copy
import json
import torch
import math
import random
from tqdm import tqdm
from augmentation import TfIdfAugmentation
from tools.utils import ExternalPreprocessor

from modelling.models import DAN, Embedding
from modelling.templates import SequenceTemplate

In [2]:
MAX_LEN = 32

In [3]:
VOCAB_PATH = '../routing/data/sberbank_embeddings/w2v_m5_w3_v300_norm_v48_vocab.txt'
W2V_MATRIX_PATH = '../routing/data/sberbank_embeddings/w2v_m5_w3_v300_norm_v48_vectors.npy'

In [4]:
with open('token2prob.json') as f:
    token2prob = json.loads(f.read())

In [5]:
with open(VOCAB_PATH) as f:
    vocab = f.read().split('\n')

In [6]:
vocab = {key: value for value, key in enumerate(vocab)}

In [7]:
index2prob = {vocab[token]: token2prob[token] for token in token2prob if token in vocab}

In [8]:
aug = TfIdfAugmentation(indexes_matrix=np.load('nearest_matrix.npy'), index2prob=index2prob)

In [9]:
with open('train.jsonl') as f:
    data = [json.loads(sample) for sample in f.read().split('\n')]

In [10]:
random.shuffle(data)

In [11]:
train, test = data[:-len(data) // 10], data[-len(data) // 10:]

In [12]:
len(train), len(test), len(train) + len(test) == len(data)

(11623, 1292, True)

In [13]:
TARGET2INDEX = {
    'ANNA.1.sales': 0,
    'ANNA.1.sbbol': 1,
    'ANNA.1.oper_support': 2
}

In [14]:
def sequence_padding(sequence, max_sequence_length, value) -> np.ndarray:

    sequence = sequence[:max_sequence_length]

    if len(sequence) < max_sequence_length:
        for _ in range((max_sequence_length - len(sequence))):
            sequence.append(value)

    sequence = np.array(sequence)

    return sequence

In [15]:
def indexing_batch(x, vocab, max_sequence_length):
    
    x = [[vocab[tok] for tok in sample if tok in vocab] for sample in x]
    
    x = np.array([sequence_padding(sample, max_sequence_length=max_sequence_length, value=0) for sample in x])
    
    return x

In [16]:
def batch_processing(batch):
    
    x = [sample['tokens'] for sample in batch]
    
    x = indexing_batch(x, vocab, MAX_LEN)
    x_aug = aug.replace_batch(copy.deepcopy(x))
    
    y = np.array([TARGET2INDEX[sample['target']] for sample in batch])
    
    x = torch.LongTensor(x)
    x_aug = torch.LongTensor(x_aug)
    y = torch.LongTensor(y)
    
    return [x, x_aug, y]

In [17]:
def loader(data, batch_size=32):

    for n_batch in range(math.ceil(len(data) / batch_size)):

        batch = data[n_batch * batch_size:(n_batch + 1) * batch_size]

        batch = batch_processing(batch)

        yield batch

In [18]:
from modelling.layers import BaseModule
from modelling.templates import SequenceTemplate

In [19]:
word_matrix = np.load(W2V_MATRIX_PATH)

In [20]:
class Model(BaseModule):
    
    def __init__(self):
        
        super().__init__()
        
        self.embedding = Embedding(vocab_size=word_matrix.shape[0],
                                   embedding_matrix=word_matrix)
        
        self.dan = DAN((300, 256), activation_function_output=torch.nn.ReLU())
        
        self.linear = torch.nn.Linear(256, 256)
        
        self.activation = torch.nn.ReLU()
        
        self.classifier = torch.nn.Linear(256, 3)
        
    def forward(self, x, x_aug):
        
        x_rep = self.embedding(x)
        x_rep = self.dan(x_rep)
        x_rep = self.linear(x_rep)
        x_rep = torch.nn.functional.log_softmax(x_rep, dim=1)
        
        with torch.no_grad():
            
            x_aug_rep = self.embedding(x_aug)
            x_aug_rep = self.dan(x_aug_rep)
            x_aug_rep = self.linear(x_aug_rep)
            x_aug_rep = torch.nn.functional.softmax(x_aug_rep, dim=1)
    
        y_pred = self.classifier(x_rep)
        
        return x_rep, x_aug_rep, y_pred

In [21]:
model = Model()

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [23]:
kl_div = torch.nn.KLDivLoss(reduction='batchmean')
cross_entropy = torch.nn.CrossEntropyLoss()

In [24]:
from tsa import TrainingSignalAnnealingScheduler

In [25]:
kl_losses = []
ce_losses = []
losses = []

l = 0.5

N_EPOCHS = 75

tsa = TrainingSignalAnnealingScheduler(total_steps=N_EPOCHS * len(train) // 32, n_classes=3, schedule_type='exp')

for n in range(N_EPOCHS):
    
    epoch_kl_losses = []
    epoch_ce_losses = []
    epoch_losses = []

    model.train()
    
    pg = tqdm(total=len(train), desc=f'Epoch: {n}')

    for x, x_aug, y in loader(train):

        optimizer.zero_grad()

        x_rep, x_aug_rep, y_pred = model(x, x_aug)

        kl_loss = kl_div(x_rep, x_aug_rep)
        epoch_kl_losses.append(kl_loss.item())
        
        y_pred, y = tsa(y_pred, y)
        
        if y_pred.size(0) == 0:
            ce_loss = 0
            epoch_ce_losses.append(ce_loss)
        else:
            ce_loss = cross_entropy(y_pred, y)
            epoch_ce_losses.append(ce_loss.item())

        loss = ce_loss + kl_loss

        loss.backward()

        optimizer.step()
        
        epoch_losses.append(loss.item())

        pg.update(x.shape[0])
        pg.set_postfix(kl_loss=epoch_kl_losses[-1], ce_loss=epoch_ce_losses[-1], loss=epoch_losses[-1])

    pg.close()
    
    test_epoch_kl_losses = []
    test_epoch_ce_losses = []
    test_epoch_losses = []

    model.eval()

    for x, x_aug, y in loader(test):

        x_rep, x_aug_rep, y_pred = model(x, x_aug)

        kl_loss = kl_div(x_rep, x_aug_rep)
        ce_loss = cross_entropy(y_pred, y)

        loss = ce_loss + kl_loss

    test_epoch_kl_losses.append(kl_loss.item())
    test_epoch_ce_losses.append(ce_loss.item())
    test_epoch_losses.append(loss.item())
    
    print('KL Train - {:.3f} | Test - {:.3f}'.format(np.mean(epoch_kl_losses), np.mean(test_epoch_kl_losses)))
    print('CE Train - {:.3f} | Test - {:.3f}'.format(np.mean(epoch_ce_losses), np.mean(test_epoch_ce_losses)))
    print('Aggregated Train - {:.3f} | Test - {:.3f}'.format(np.mean(epoch_losses), np.mean(test_epoch_losses)))
    
    kl_losses.extend(copy.deepcopy(epoch_kl_losses))
    ce_losses.extend(copy.deepcopy(epoch_ce_losses))
    losses.extend(copy.deepcopy(epoch_losses))

Epoch: 1:   7%|▋         | 864/11623 [00:00<00:02, 4523.23it/s, ce_loss=1.13, kl_loss=0.000166, loss=1.13]

KL Train - 0.000 | Test - 0.000
CE Train - 1.157 | Test - 1.059
Aggregated Train - 1.157 | Test - 1.059


Epoch: 2:   8%|▊         | 896/11623 [00:00<00:02, 4578.58it/s, ce_loss=1.14, kl_loss=0.000221, loss=1.14]

KL Train - 0.000 | Test - 0.000
CE Train - 1.108 | Test - 1.056
Aggregated Train - 1.109 | Test - 1.056


Epoch: 3:   8%|▊         | 896/11623 [00:00<00:02, 4528.47it/s, ce_loss=1.17, kl_loss=0.000253, loss=1.17]

...

KL Train - 0.012 | Test - 0.015
CE Train - 0.412 | Test - 0.480
Aggregated Train - 0.424 | Test - 0.495


Epoch: 74:   8%|▊         | 896/11623 [00:00<00:02, 4584.97it/s, ce_loss=0.321, kl_loss=0.0179, loss=0.339] 

KL Train - 0.013 | Test - 0.005
CE Train - 0.364 | Test - 0.469
Aggregated Train - 0.377 | Test - 0.474


Epoch: 74: 100%|██████████| 11623/11623 [00:02<00:00, 4714.55it/s, ce_loss=0.123, kl_loss=0.0135, loss=0.136] 


KL Train - 0.013 | Test - 0.015
CE Train - 0.301 | Test - 0.459
Aggregated Train - 0.315 | Test - 0.474


In [26]:
tsa.tsa.current_prob

1.0

In [25]:
class Model(BaseModule):
    
    def __init__(self):
        
        super().__init__()
        
        self.embedding = Embedding(vocab_size=word_matrix.shape[0],
                                   embedding_matrix=word_matrix)
        
        self.dan = DAN((300, 256), activation_function_output=torch.nn.ReLU())
        
        self.linear = torch.nn.Linear(256, 256)
        
        self.activation = torch.nn.ReLU()
        
        self.classifier = torch.nn.Linear(256, 3)
        
    def forward(self, x):
        
        x_rep = self.embedding(x)
        x_rep = self.dan(x_rep)
        x_rep = self.linear(x_rep)
        x_rep = torch.nn.functional.log_softmax(x_rep, dim=1)
    
        y_pred = self.classifier(x_rep)
        
        return y_pred

In [26]:
model = Model()

In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [28]:
losses = []

l = 0.5

N_EPOCHS = 75

tsa = TrainingSignalAnnealingScheduler(total_steps=N_EPOCHS * len(train) // 32, n_classes=3, schedule_type='exp')

for n in range(N_EPOCHS):
    
    epoch_losses = []

    model.train()
    
    pg = tqdm(total=len(train), desc=f'Epoch: {n}')

    for x, _, y in loader(train):

        optimizer.zero_grad()

        y_pred = model(x)
        
        y_pred, y = tsa(y_pred, y)
        
        if y_pred.size(0) == 0:
            loss = 0
            epoch_losses.append(loss)
        else:
            loss = cross_entropy(y_pred, y)
            epoch_losses.append(loss.item())
            
            loss.backward()

            optimizer.step()

#         loss = cross_entropy(y_pred, y)

        pg.update(x.shape[0])
        pg.set_postfix(loss=epoch_losses[-1])

    pg.close()

    test_epoch_losses = []

    model.eval()

    for x, x_aug, y in loader(test):

        y_pred = model(x)

        loss = cross_entropy(y_pred, y)

    test_epoch_losses.append(loss.item())
    
    print('Loss Train - {:.3f} | Test - {:.3f}'.format(np.mean(epoch_losses), np.mean(test_epoch_losses)))

    losses.extend(copy.deepcopy(epoch_losses))

Epoch: 1:   9%|▊         | 992/11623 [00:00<00:02, 5091.79it/s, loss=1.1] 

Loss Train - 1.209 | Test - 1.026


Epoch: 2:   9%|▊         | 992/11623 [00:00<00:02, 5113.96it/s, loss=1.12]

Loss Train - 1.124 | Test - 1.032


Epoch: 3:   9%|▊         | 992/11623 [00:00<00:02, 5146.64it/s, loss=1.12]

...

Loss Train - 0.416 | Test - 0.494


Epoch: 74:   9%|▊         | 992/11623 [00:00<00:02, 5094.32it/s, loss=0.316]

Loss Train - 0.367 | Test - 0.495


Epoch: 74: 100%|██████████| 11623/11623 [00:02<00:00, 5007.52it/s, loss=0.0707]


Loss Train - 0.306 | Test - 0.494
